In [1]:
import sys, threading
sys.setrecursionlimit(800000)
threading.stack_size(67108864)
    
import networkx as nx

# STANDARD CODE FOR DEPTH-FIRST SEARCH

def DFS_Original(graph,s,explored=[]):
    # This auxiliary function performs Depth-First search on a graph called 'graph' assuming that one starts at
    # the node 's' and the nodes in the list 'explored' have already been explored.
    
    # INPUT: A networkx graph called 'graph', a starting node 's' from which we start the Depth-First search, and a
    # list called 'explored' containing the nodes in 'graph' that have already been explored.
    # OUTPUT: A set containing the nodes in 'graph' which can be accesed from starting node 's'.
    
    explored.append(s)   
    for x in list(graph.edges([s])):
        if x[1] not in explored:
            DFS_Original(graph,x[1],explored)
    if set(list(graph.neighbors(s))) <= set(explored):
        return set(explored)
    
# CHECK OF THE FUNCTION 'DFS_Original'

# A networkx graph 'G_Ex1' is defined to test the function
G_Ex1 = nx.DiGraph()
G_Ex1.add_nodes_from(['s','v','w','t'])
G_Ex1.add_edge('s','v')
G_Ex1.add_edge('s','w')
G_Ex1.add_edge('v','t')
G_Ex1.add_edge('w','t')

# The function is applied to the graph 'G_Ex1'
DFS_Original(G_Ex1,'v',[])

{'t', 'v'}

In [2]:
# STANDARD CODE FOR TOPOLOGICAL SORT

def DFS_Loop(graph): # This is the main function for topological sort
    
    # INPUT: A networkx graph called 'graph'.
    # OUTPUT A list containing the topological sort.
    
    explored=[]
    f=[]
    current_label=len(list(graph.nodes()))
    
    for v in reversed(list(graph.nodes())):
        if v not in explored:
            DFS(graph,v,explored,f)
    return f

def DFS(graph,s,explored,f): # This is an auxiliary function used in 'DFS_Loop'
    
    # INPUT: A networkx graph 'graph', a starting node 's' of the graph, a list 'explored' containing the labels of
    # the vertices already explored, a list 'f'.
    # OUTPUT: The updated list 'f'.

    explored.append(s)  
    for x in list(graph.edges([s])):
        if x[1] not in explored:
            DFS(graph,x[1],explored,f)
    if set(list(graph.neighbors(s))) <= set(explored):
        f.insert(0,s)
        return f

In [3]:
# CHECK OF TOPOLOGICAL SORT CODE USING NETWORKX'S IMPLEMENTATION AND THE FUNCTION 'DFS_Loop'

print('Check of Topological Sort:')
print("My code: "+str(DFS_Loop(G_Ex1)))
print("NetworkX: "+str(list(nx.topological_sort(G_Ex1))))
print(list(nx.all_topological_sorts(G_Ex1)))
print('Does it work? '+str(DFS_Loop(G_Ex1) in list(nx.all_topological_sorts(G_Ex1))))
print('')

# CHECK OF THE FINISHING TIME USING 'DFS_Loop'

# A networkx graph 'G_Ex2' is defined to test the function:
G_Ex2 = nx.DiGraph()
G_Ex2.add_nodes_from([1,2,3,4,5,6,7,8,9])
G_Ex2.add_edge(7,1)
G_Ex2.add_edge(5,2)
G_Ex2.add_edge(9,3)
G_Ex2.add_edge(1,4)
G_Ex2.add_edge(8,5)
G_Ex2.add_edge(3,6)
G_Ex2.add_edge(8,6)
G_Ex2.add_edge(4,7)
G_Ex2.add_edge(9,7)
G_Ex2.add_edge(2,8)
G_Ex2.add_edge(6,9)
G_Ex2_rev=G_Ex2.reverse(copy=True) # This line constructs the reverse graph.
print('Check of Finishing Time:')
#print("NetworkX: "+str(list(nx.topological_sort(G_Ex2))))
print("My code's finishing times (left is largest, right is smallest): "+str(DFS_Loop(G_Ex2_rev)))

Check of Topological Sort:
My code: ['s', 'v', 'w', 't']
NetworkX: ['s', 'w', 'v', 't']
[['s', 'w', 'v', 't'], ['s', 'v', 'w', 't']]
Does it work? True

Check of Finishing Time:
My code's finishing times (left is largest, right is smallest): [7, 4, 1, 9, 6, 8, 2, 5, 3]


In [4]:
# KOSARAJU'S ALGORITHM FOR STRONGLY CONNECTED COMPONENTS

def Kosaraju(graph):
    # This function implements Kosaraju's algorithm to find the strongly connected components of a graph. This
    # implementation is using a double recursive Depth-First search.
    
    # INPUT: A networkx graph called 'graph'.
    # OUTPUT: A list containing as entries the strongly connected components of the graph. Each entry of the list 
    # is a set which has as elements the nodes of the strongly connected component. 
    
    n=len(graph)
    cc=[]
    explored=[]
    graph_rev=graph.reverse(copy=True)
    finishing_time=DFS_Loop(graph_rev) #1st Depth-First search

    dic={}
    for i in range(n):
        dic[finishing_time[i]]=n-i
    graph2=nx.relabel_nodes(graph,dic, copy=True)

    for x in sorted(list(graph2.nodes()),reverse=True):
        if x not in explored:
            explored_aux=set(explored)
            y=set(DFS_Original(graph2,x,explored))-explored_aux #2nd Depth-First search
            yy=[]
            for x in y:
                yy.append(finishing_time[n-x])
            cc.append(set(yy))
    return cc

# CHECK OF KOSARAJU'S ALGORITHM FOR STRONGLY CONNECTED COMPONENTS 

# The results of the networkx implementation and the function 'Kosaraju' are compared for the cases of graphs
# 'G_Ex1' and 'G_Ex2':
print('Check using the graph G_Ex1:')
print('Networkx: '+str(list(nx.strongly_connected_components(G_Ex1))))
print('My code: '+str(Kosaraju(G_Ex1)))
print(' ')
print('Check using the graph G_Ex2:')
print('Networkx: '+str(list(nx.strongly_connected_components(G_Ex2))))
print('My code: '+str(Kosaraju(G_Ex2)))

Check using the graph G_Ex1:
Networkx: [{'t'}, {'v'}, {'w'}, {'s'}]
My code: [{'t'}, {'w'}, {'v'}, {'s'}]
 
Check using the graph G_Ex2:
Networkx: [{1, 4, 7}, {9, 3, 6}, {8, 2, 5}]
My code: [{1, 4, 7}, {9, 3, 6}, {8, 2, 5}]


In [5]:
# FINAL ANSWER TO HOMEWORK

# LOADING 'SCC.txt' INTO A NETWORKX GRAPH 'G':

# The file 'SCC.txt' contains the edges of a directed graph. In it the vertices are labeled as positive integers 
# from 1 to 875714. Every row indicates an edge, the vertex label in the first column is the tail and the vertex 
# label in the second column is the head.

G = nx.read_edgelist('SCC.txt',create_using=nx.DiGraph,nodetype=int)

# LOOKING FOR THE NUMBER OF NODES IN THE 5 LARGEST STRONGLY CONNECTED COMPONENTS OF THE GRAPH 'G'

#Using networkx:
q=[len(x) for x in list(nx.strongly_connected_components(G))]
q.sort(reverse = True)
q[0:5]

# Using the function 'Kosaraju' defined above (This takes a very long time):
#Kosaraju(G)
#Q=[len(x) for x in list(Kosaraju(G))]
#Q.sort(reverse = True)
#Q[0:5]

[434821, 968, 459, 313, 211]